In [0]:
#REFERENCE: http://www.jacketheredge.com/mnist-cifar10_tutorial-modded-v3.html 

In [0]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import torchvision.transforms as transforms


import copy
#from base_test import BaseT|est
#import digits_model
import numpy as np
import matplotlib.pyplot as plt
#import data

import torch
import torchvision
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import time
import os
#reference:  https://nextjournal.com/gkoehler/pytorch-mnist

In [0]:
transform = transforms.Compose([
transforms.Pad(2),
transforms.ToTensor(),
])

target_train_set = torchvision.datasets.MNIST(root='./MNIST/', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(target_train_set, batch_size=4, shuffle=True, num_workers=2)

target_test_set = torchvision.datasets.MNIST(root='./MNIST/', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(target_test_set, batch_size=4, shuffle=True, num_workers=2)



classes = ('0','1','2','3','4','5','6','7','8','9')


In [0]:
target_train_set.data.shape, target_test_set.data.shape

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [0]:
for epoch in range(2):  # loop over the dataset multiple times 

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    correct = 0
    total = 0
    for data in testloader:
        images, labels = data
        outputs = net(Variable(images))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    for data in testloader:
        images, labels = data
        outputs = net(Variable(images))
        _, predicted = torch.max(outputs.data, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i]
            class_total[label] += 1

    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))
    
print('Finished Training')

[1,  2000] loss: 1.608
[1,  4000] loss: 0.296
[1,  6000] loss: 0.164
[1,  8000] loss: 0.121
[1, 10000] loss: 0.111
[1, 12000] loss: 0.100
[1, 14000] loss: 0.096
Accuracy of the network on the 10000 test images: 97 %
Accuracy of     0 :  0 %
Accuracy of     1 :  0 %
Accuracy of     2 : 29 %
Accuracy of     3 :  0 %
Accuracy of     4 :  0 %
Accuracy of     5 :  0 %
Accuracy of     6 :  0 %
Accuracy of     7 : 48 %
Accuracy of     8 :  0 %
Accuracy of     9 :  0 %
[2,  2000] loss: 0.076
[2,  4000] loss: 0.063
[2,  6000] loss: 0.074
[2,  8000] loss: 0.062
[2, 10000] loss: 0.066
[2, 12000] loss: 0.062
[2, 14000] loss: 0.057
Accuracy of the network on the 10000 test images: 98 %
Accuracy of     0 :  0 %
Accuracy of     1 :  1 %
Accuracy of     2 : 31 %
Accuracy of     3 :  0 %
Accuracy of     4 :  0 %
Accuracy of     5 :  0 %
Accuracy of     6 :  0 %
Accuracy of     7 : 29 %
Accuracy of     8 :  0 %
Accuracy of     9 :  0 %
Finished Training


In [0]:
net.type(torch.cuda.FloatTensor)
summary(nn.Sequential(net),(1,32,32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
         MaxPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         MaxPool2d-4             [-1, 16, 5, 5]               0
            Linear-5                  [-1, 120]          48,120
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
               Net-8                   [-1, 10]               0
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.30
----------------------------------------------------------------


In [0]:
from collections import OrderedDict
dic={}
dic["state_dict"]=OrderedDict
dic["state_dict"]=net.state_dict()
dic

{'state_dict': OrderedDict([('conv1.weight',
               tensor([[[[ 0.1361,  0.3789,  0.2161, -0.0394,  0.0123],
                         [ 0.0861,  0.2112,  0.1671,  0.1526, -0.0279],
                         [ 0.3235, -0.0077, -0.1250, -0.2238,  0.0043],
                         [-0.0916, -0.1447, -0.2219,  0.0827, -0.2360],
                         [-0.2178, -0.1025, -0.1481, -0.0906, -0.1281]]],
               
               
                       [[[ 0.4718,  0.7005,  0.6254,  0.4367,  0.5363],
                         [ 0.3611,  0.2816,  0.3696,  0.1910,  0.0978],
                         [-0.2121, -0.2383, -0.2356, -0.1001,  0.0291],
                         [-0.5640, -0.4498, -0.3845, -0.4601, -0.2174],
                         [-0.2048, -0.2831, -0.2519, -0.0083, -0.1310]]],
               
               
                       [[[ 0.1124,  0.1828, -0.0331,  0.1350,  0.0717],
                         [ 0.2411,  0.2853,  0.3014,  0.3856,  0.3754],
                       

In [0]:
#torch.save(dic, "./sample_data/MNIST_clf_cifar10.tar")